In [6]:
# numpy and pandas imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# ML Libraries for scikit
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn import tree
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,\
                            precision_recall_curve,precision_score,recall_score,roc_auc_score,roc_curve,\
                            matthews_corrcoef, f1_score, make_scorer, auc
from scipy.stats import skew

#  Recursive Feature Elimination
from sklearn.feature_selection import RFE
        
# Python imports
from math import log, sqrt
import re
import collections
import warnings

# decision tree visualization related imports
import pydotplus
import graphviz
import dtale
from IPython.display import Image

warnings.filterwarnings('ignore')

pd.options.display.max_columns = 400

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
pd.set_option('display.max_rows', None)

In [3]:
df = train.copy()

In [4]:
df.head(10)

,Unnamed: 0,loan_id,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13
0,81041,861482495205,Y,"Martinez, Duffy and Bird",3.375,272000,180,2012-01-01,03/2012,72,2.0,31.0,770.0,B12,0.0,786.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,51021,173139140902,Y,"Swanson, Newton and Miller",4.250,371000,360,2012-01-01,03/2012,95,1.0,45.0,727.0,C86,30.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,83866,481942560913,Y,OTHER,4.500,90000,360,2012-01-01,03/2012,69,1.0,39.0,726.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,92311,676780245612,Y,"Turner, Baldwin and Rhodes",4.500,47000,240,2012-02-01,04/2012,75,1.0,43.0,688.0,B12,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,72274,433043514697,X,OTHER,4.875,177000,360,2012-01-01,03/2012,80,2.0,13.0,675.0,C86,0.0,672.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,16878,158075925162,Y,Browning-Hart,3.875,168000,360,2012-01-01,03/2012,80,1.0,24.0,744.0,C86,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,44712,275134357431,X,OTHER,4.000,100000,360,2012-02-01,04/2012,87,1.0,40.0,817.0,C86,25.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,25439,662971099269,X,OTHER,3.875,194000,360,2012-02-01,04/2012,49,2.0,42.0,798.0,A23,0.0,811.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,46417,951861918664,X,Browning-Hart,3.875,415000,360,2012-01-01,03/2012,73,1.0,28.0,792.0,A23,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,71591,465552456707,Z,"Turner, Baldwin and Rhodes",3.875,200000,360,2012-03-01,05/2012,77,2.0,23.0,793.0,A23,0.0,793.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
df['interest_rate'].describe()

count    92846.000000
mean         3.869010
std          0.461126
min          2.250000
25%          3.500000
50%          3.875000
75%          4.125000
max          6.750000
Name: interest_rate, dtype: float64

In [15]:
df['first_payment_date'].value_counts()

2012-02-01    41951
2012-01-01    39235
2012-03-01    11660
Name: origination_date, dtype: int64

In [19]:
df['January'] = (df['origination_date']=='2012-01-01').astype('int')
df['February'] = (df['origination_date']=='2012-02-01').astype('int')
df['March'] = (df['origination_date']=='2012-03-01').astype('int')
df.drop('origination_date',axis=1,inplace=True)

In [ ]:
def date_cat(a):
    
    df['January'] = (a=='2012-01-01').astype('int')
    df['February'] = (a=='2012-02-01').astype('int')
    df['March'] = (a=='2012-03-01').astype('int')
    df.drop('origination_date',axis=1,inplace=True)
    

In [20]:
df.head(10)

,Unnamed: 0,loan_id,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,January,February,March
0,81041,861482495205,Y,"Martinez, Duffy and Bird",3.375,272000,180,03/2012,72,2.0,31.0,770.0,B12,0.0,786.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,51021,173139140902,Y,"Swanson, Newton and Miller",4.250,371000,360,03/2012,95,1.0,45.0,727.0,C86,30.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,83866,481942560913,Y,OTHER,4.500,90000,360,03/2012,69,1.0,39.0,726.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,92311,676780245612,Y,"Turner, Baldwin and Rhodes",4.500,47000,240,04/2012,75,1.0,43.0,688.0,B12,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
4,72274,433043514697,X,OTHER,4.875,177000,360,03/2012,80,2.0,13.0,675.0,C86,0.0,672.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5,16878,158075925162,Y,Browning-Hart,3.875,168000,360,03/2012,80,1.0,24.0,744.0,C86,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
6,44712,275134357431,X,OTHER,4.000,100000,360,04/2012,87,1.0,40.0,817.0,C86,25.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
7,25439,662971099269,X,OTHER,3.875,194000,360,04/2012,49,2.0,42.0,798.0,A23,0.0,811.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
8,46417,951861918664,X,Browning-Hart,3.875,415000,360,03/2012,73,1.0,28.0,792.0,A23,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
9,71591,465552456707,Z,"Turner, Baldwin and Rhodes",3.875,200000,360,05/2012,77,2.0,23.0,793.0,A23,0.0,793.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [21]:
df['first_payment_date'].value_counts()

04/2012    42282
03/2012    38138
05/2012    12000
02/2012      426
Name: first_payment_date, dtype: int64

In [22]:
df['fpay_1'] = (df['first_payment_date']=='02/2012').astype('int')
df['fpay_2'] = (df['first_payment_date']=='03/2012').astype('int')
df['fpay3'] = (df['first_payment_date']=='04/2012').astype('int')
df['fpay4'] = (df['first_payment_date']=='05/2012').astype('int')
df.drop('first_payment_date',axis=1,inplace=True)

In [23]:
df.head()

,Unnamed: 0,loan_id,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,January,February,March,fpay_1,fpay_2,fpay3,fpay4
0,81041,861482495205,Y,"Martinez, Duffy and Bird",3.375,272000,180,72,2.0,31.0,770.0,B12,0.0,786.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
1,51021,173139140902,Y,"Swanson, Newton and Miller",4.250,371000,360,95,1.0,45.0,727.0,C86,30.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
2,83866,481942560913,Y,OTHER,4.500,90000,360,69,1.0,39.0,726.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
3,92311,676780245612,Y,"Turner, Baldwin and Rhodes",4.500,47000,240,75,1.0,43.0,688.0,B12,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,72274,433043514697,X,OTHER,4.875,177000,360,80,2.0,13.0,675.0,C86,0.0,672.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0


In [ ]:
pd.get_dummies(df['financial_institution'])